This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_010_Li_order4_off__Li_100_slab_heavy
- Initial lattice height: 66.24 Å
- Number of atoms: 536

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 13.5 Å
- Li top: frozen top 13.5 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 14-07-2025


In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = "/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_010_Li_order4_off__Li_100_slab_heavy.cif"  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_Li_order4_off__Li_100_slab_heavy
- Initial lattice height: 66.24 Å
- Number of atoms: 536


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 48 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [ ]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file("/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_010_Li_order4_off__Li_100_slab_heavy.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 19.86 Å
Lowest Li slab atom: 39.40 Å
LLZO top z: 35.40 Å
Li penetration into LLZO: -4.00 Å


In [ ]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read("/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_010_Li_order4_off__Li_100_slab_heavy.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (freeze_thickness_llzo)  # in Å
llzo_z_threshold = z_min + freeze_thickness
li_z_threshold = z_max - freeze_thickness

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)



Freezing 415 atoms out of 536
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_54570/706244439.py:26: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 19:50:03    -2127.541367        0.753929
SciPyFminCG:    1 19:50:04    -2130.280085        5.299298
SciPyFminCG:    2 19:50:06    -2150.828201       10.744150
SciPyFminCG:    3 19:50:07    -2153.393761        8.573008
SciPyFminCG:    4 19:50:07    -2164.855713        4.229665
SciPyFminCG:    5 19:50:08    -2170.750519        1.665468
SciPyFminCG:    6 19:50:09    -2172.425110        2.088161
SciPyFminCG:    7 19:50:10    -2173.947372        0.715406
SciPyFminCG:    8 19:50:11    -2174.934185        2.318305
SciPyFminCG:    9 19:50:12    -2175.629375        2.063509
SciPyFminCG:   10 19:50:12    -2176.719444        1.863614
SciPyFminCG:   11 19:50:13    -2177.181541        1.148508
SciPyFminCG:   12 19:50:14    -2177.681465        1.074727
SciPyFminCG:   13 19:50:14    -2178.075577        1.209065
SciPyFminCG:   14 19:50:16    -2178.868401        2.191883
SciPyFminCG:   15 19:50:16    -2179.873360        2.332675

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_54570/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 19:51:06    -2254.858082        0.148592
FIRE:    1 19:51:06    -2254.860893        0.116538
FIRE:    2 19:51:06    -2254.866005        0.085804
FIRE:    3 19:51:06    -2254.871372        0.075637
FIRE:    4 19:51:07    -2254.876995        0.074435
FIRE:    5 19:51:07    -2254.882107        0.090921
FIRE:    6 19:51:08    -2254.887218        0.085918
FIRE:    7 19:51:08    -2254.892330        0.073052
FIRE:    8 19:51:08    -2254.897697        0.076297
FIRE:    9 19:51:09    -2254.903320        0.074244
FIRE:   10 19:51:09    -2254.909199        0.072273
FIRE:   11 19:51:09    -2254.916355        0.077621
FIRE:   12 19:51:10    -2254.924534        0.085653
FIRE:   13 19:51:10    -2254.934246        0.087056
FIRE:   14 19:51:11    -2254.946259        0.090867
FIRE:   15 19:51:11    -2254.961082        0.119570
FIRE:   16 19:51:11    -2254.979485        0.116052
FIRE:   17 19:51:12    -2255.000954        0.081164
FIRE:   18 19:

In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"{structure_name}.cif", final_structure_ase)
write(f"{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [14]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file("/home/mehuldarak/summer/llzo_li_balanced_sliced/LLZO_001_Zr_code93_sto__Li_100_slab_heavy.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 19.86 Å
Lowest Li slab atom: 39.40 Å
LLZO top z: 35.40 Å
Li penetration into LLZO: -4.00 Å
